In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

C:\Users\hp\AppData\Local\Temp\ipykernel_11900\2644893652.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Set random seeds for reproducibility
#np.random.seed(42)
#tf.random.set_seed(42)

In [3]:
# Load and preprocess the data
df = pd.read_csv('IESCO_dataset.csv')
df.dropna(inplace=True)
s1 = MinMaxScaler(feature_range=(0, 1))
s2 = MinMaxScaler(feature_range=(0, 1))

In [4]:
df

,Hour,DryBulb,DewPnt,target_variable,Humidity
0,1,37,25,827,87.5
1,2,37,25,798,88.0
2,3,39,24,778,88.0
3,4,38,22,778,88.0
4,5,37,20,770,88.0
...,...,...,...,...,...
13123,20,58,38,1095,50.0
13124,21,56,39,1092,47.5
13125,22,54,39,1045,45.0
13126,23,53,38,923,45.0


In [5]:
# Selecting only the target variable and scaling
ys = s2.fit_transform(df[['target_variable']])

In [6]:
# Scaling the input features
Xs = s1.fit_transform(df.drop(columns=['target_variable']))
window = 1
num_features = Xs.shape[1]

In [7]:
# Reshape the input features to have shape (None, 5, num_features)
Xs_reshaped = np.array([Xs[i:i+window] for i in range(len(Xs)-window+1)])

# Slice the target variable to match the length of Xs_reshaped
ys_sliced = ys[window-1:]

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(Xs_reshaped, ys_sliced, test_size=0.2, shuffle=False)

In [9]:
def dual_attention_lstm(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    lstm_out = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)
    attention1 = tf.keras.layers.Attention()([lstm_out, lstm_out])
    attention1 = tf.keras.layers.GlobalAveragePooling1D()(attention1)
    attention2 = tf.keras.layers.Attention()([attention1, lstm_out])
    attention2 = tf.keras.layers.GlobalAveragePooling1D()(attention2)
    outputs = tf.keras.layers.Dense(1, activation='linear')(attention2)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = dual_attention_lstm((X_train.shape[1], num_features))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [10]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32)

loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test MAE: {mae}')

y_pred = s2.inverse_transform(model.predict(X_test))
y_test = s2.inverse_transform(y_test)

Epoch 1/200


329/329 [==============================] - 7s 6ms/step - loss: 0.0240 - mae: 0.1199
Epoch 2/200
329/329 [==============================] - 2s 5ms/step - loss: 0.0164 - mae: 0.1020
Epoch 3/200
329/329 [==============================] - 2s 6ms/step - loss: 0.0161 - mae: 0.1008
Epoch 4/200
329/329 [==============================] - 4s 11ms/step - loss: 0.0156 - mae: 0.0993
Epoch 5/200
329/329 [==============================] - 2s 7ms/step - loss: 0.0148 - mae: 0.0963
Epoch 6/200
329/329 [==============================] - 3s 9ms/step - loss: 0.0136 - mae: 0.0914
Epoch 7/200
329/329 [==============================] - 2s 7ms/step - loss: 0.0127 - mae: 0.0876
Epoch 8/200
329/329 [==============================] - 2s 7ms/step - loss: 0.0125 - mae: 0.0864
Epoch 9/200
329/329 [==============================] - 2s 7ms/step - loss: 0.0123 - mae: 0.0856
Epoch 10/200
329/329 [==============================] - 3s 8ms/step - loss: 0.0122 - mae: 0.0852
Epoch 11/200
329/329 [==============

In [11]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
mse = mean_squared_error(y_test, y_pred)
nrmse = (rmse / np.mean(y_pred)) * 100

print("RMSE: ", rmse)
print("MAE", mae)
print("MAPE", mape)
print("MSE", mse)
print("NRMSE", nrmse)

RMSE:  164.567131732261
MAE 129.65379925021122
MAPE 12.974108131724982
MSE 27082.340846583345
NRMSE 15.488441671196304


In [12]:
y_pred= pd.DataFrame(y_pred)
y_test= pd.DataFrame(y_test)

In [15]:
result_df = pd.concat([y_pred, y_test], axis=1)
result_df.to_csv(r"C:\Users\hp\Desktop\MODELS\IESCO\DA-LSTM\DA-LSTM_output.csv", index=False)